In [5]:
import re
import os
import pandas as pd
import random
import json

/home/prevot/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [6]:
# MODIFYING GENUINE EXAMPLES
def move_fp(row):
    utt_as_list = row['text'].split()
    filtered_utt = [t for t in utt_as_list if t not in FP_ITEMS]
    items_to_move = [t for t in utt_as_list if t in FP_ITEMS]
    new_utt = utt_as_list
    while new_utt == utt_as_list:
        new_utt = filtered_utt
        for item in items_to_move:
            insert_location = random.randrange(2, len(new_utt)-2)
            new_utt.insert(insert_location,item) 
    return ' '.join(new_utt)


def move_sp(row):
    utt_as_list = row['text'].split()
    filtered_utt = [t for t in utt_as_list if t not in SP_ITEMS]
    items_to_move = [t for t in utt_as_list if t in SP_ITEMS]
    new_utt = utt_as_list
    while new_utt == utt_as_list:
        new_utt = filtered_utt
        for item in items_to_move:
            insert_location = random.randrange(2, len(new_utt)-2)
            new_utt.insert(insert_location,item) 
    return ' '.join(new_utt)


def move_rep(row):
    utt_as_list = row['text'].split()

    tmp = []
    i = 0
    while i<len(utt_as_list)-1:
        if utt_as_list[i] != utt_as_list[i+1]:
            tmp.append(utt_as_list[i])
            i+=1
        else:
            original_repeat_location = i
            i+=1
    tmp.append(utt_as_list[-1])
    location = original_repeat_location
    while location == original_repeat_location:
        location = random.randint(0, len(tmp))

    tmp.insert(location, tmp[location-1])
    
    return ' '.join(tmp)


def replace_fp(row):
    utt_as_list = row['text'].split()
    targets = [x for x in utt_as_list if x not in FP_ITEMS + ['#', '*', ',']]
    for fp in FP_ITEMS:
        utt_as_list = [re.sub("^"+fp+"$", random.choice(NOT_FP), x) for x in utt_as_list]
    return ' '.join(utt_as_list)

def replace_sp(row):
    utt_as_list = row['text'].split()
    targets = [x for x in utt_as_list if x not in SP_ITEMS + ['#', '*', ',']]
    for fp in SP_ITEMS:
        utt_as_list = [re.sub("^"+fp+"$", random.choice(NOT_SP), x) for x in utt_as_list]
    return ' '.join(utt_as_list)


def shuffle_utterance(row):
    utt_as_list = row['text'].split()
    random.shuffle(utt_as_list)
    return ' '.join(utt_as_list)


In [7]:
### BASE
N_EXPS = 10

## French
### Filled Pauses

In [11]:
### FILLED PAUSES 

folder = './data_benchmark/disfl/'
if not os.path.exists(folder):
    os.makedirs(folder)


# FR
LAN = 'fr'
FILE_FP = 'data_corpus/french_fp_base_500_checked.txt'
FP_ITEMS= ['euh']


FP_EXAMPLES = [u[:-1] for u in open(FILE_FP,'r', encoding = 'utf-8').readlines()]

with open(FILE_FP, 'r', encoding = 'utf-8') as f:
    NOT_FP = [x for x in f.read().split() if x not in FP_ITEMS + ['#', '*', ',']]

FP_DF = pd.DataFrame(FP_EXAMPLES,columns=['text'])


for i in range(N_EXPS):
    FP_DF['moved_fp'] = FP_DF.apply(move_fp,axis=1)
    #FP_DF['removed_fp'] = FP_DF.apply(remove_fp,axis=1)
    FP_DF['replaced_fp'] = FP_DF.apply(replace_fp,axis=1)
    FP_DF['shuffled'] = FP_DF.apply(shuffle_utterance,axis=1)
    
    exp_tag = LAN+'_'+str(i)
    
    # SHUFFLED    
    result = []
    cnt=0
    for i,row in FP_DF.iterrows():
        item_shuffled = {"sentence_good":row['text'],"sentence_bad":row['shuffled'], "field": "disfluencies", 
                         "linguistics_term": "shuffled_"+LAN, "UID": "shuffled_"+exp_tag, "simple_LM_method": True, 
                         "one_prefix_method": False, "two_prefix_method": False, "lexically_identical": True, 
                         "pair_id": str(cnt)}
        result.append(item_shuffled)
        cnt +=1
            
    with open(folder + 'shuffled_' + exp_tag + '.json', 'w', encoding='utf-8') as outfile:
        for entry in result:
            json.dump(entry, outfile, ensure_ascii=False)
            outfile.write('\n')
        
    # MOVED    
    result = []
    cnt=0
    for i,row in FP_DF.iterrows():
        item_moved = {"sentence_good":row['text'], "sentence_bad":row['moved_fp'], "field": "disfluencies", 
                         "linguistics_term": "moved_filler_"+LAN, "UID": "moved_filler_"+exp_tag, "simple_LM_method": True,
                         "one_prefix_method": False, "two_prefix_method": False, 
                         "lexically_identical": True, "pair_id": str(cnt)}
        result.append(item_moved)
        cnt +=1
            
    with open(folder + 'moved_filler_' + exp_tag + '.json', 'w', encoding='utf-8') as outfile:
        for entry in result:
            json.dump(entry, outfile, ensure_ascii=False)
            outfile.write('\n')
            
    #REPLACED
    result = []
    cnt=0
    for i,row in FP_DF.iterrows():
        item_removed = {"sentence_good":row['text'],"sentence_bad":row['replaced_fp'], "field": "disfluencies", 
                        "linguistics_term": "replaced_filler_"+LAN, "UID": "replaced_filler_"+exp_tag, "simple_LM_method": True,
                        "one_prefix_method": False, "two_prefix_method": False, 
                        "lexically_identical": False, "pair_id": str(cnt)}
        result.append(item_removed)
        cnt +=1
        
    with open(folder + 'replaced_filler_' + exp_tag + '.json', 'w', encoding='utf-8') as outfile:
        for entry in result:
            json.dump(entry, outfile, ensure_ascii=False)
            outfile.write('\n')

### Silent Pauses

In [14]:
### SILENT PAUSES 

folder = './data_benchmark/disfl/'
if not os.path.exists(folder):
    os.makedirs(folder)


# FR
LAN = 'fr'
FILE_SP = 'data_corpus/french_sp_base_500_checked.txt'
SP_ITEMS= ['#']

SP_EXAMPLES = [u[:-1] for u in open(FILE_SP,'r', encoding = 'utf-8').readlines()]

with open(FILE_SP, 'r', encoding = 'utf-8') as f:
    NOT_SP = [x for x in f.read().split() if x not in SP_ITEMS + ['#', '*', ',']]

SP_DF = pd.DataFrame(SP_EXAMPLES,columns=['text'])


for i in range(N_EXPS):
    SP_DF['moved_sp'] = SP_DF.apply(move_sp,axis=1)
    #SP_DF['removed_sp'] = SP_DF.apply(remove_sp,axis=1)
    SP_DF['replaced_sp'] = SP_DF.apply(replace_sp,axis=1)
    SP_DF['shuffled'] = SP_DF.apply(shuffle_utterance,axis=1)
    
    exp_tag = LAN+'_'+str(i)
    
    # SHUFFLED    
    result = []
    cnt=0
    for i,row in SP_DF.iterrows():
        item_shuffled = {"sentence_good":row['text'],"sentence_bad":row['shuffled'], "field": "disfluencies", 
                         "linguistics_term": "shuffled_"+LAN, "UID": "shuffled_"+exp_tag, "simple_LM_method": True, 
                         "one_prefix_method": False, "two_prefix_method": False, "lexically_identical": True, 
                         "pair_id": str(cnt)}
        result.append(item_shuffled)
        cnt +=1
            
    with open(folder + 'shuffled_pause_' + exp_tag + '.json', 'w', encoding='utf-8') as outfile:
        for entry in result:
            json.dump(entry, outfile, ensure_ascii=False)
            outfile.write('\n')
        
    # MOVED    
    result = []
    cnt=0
    for i,row in SP_DF.iterrows():
        item_moved = {"sentence_good":row['text'], "sentence_bad":row['moved_sp'], "field": "disfluencies", 
                         "linguistics_term": "moved_pause_"+LAN, "UID": "moved_pause_"+exp_tag, "simple_LM_method": True,
                         "one_prefix_method": False, "two_prefix_method": False, 
                         "lexically_identical": True, "pair_id": str(cnt)}
        result.append(item_moved)
        cnt +=1
            
    with open(folder + 'moved_pause_' + exp_tag + '.json', 'w', encoding='utf-8') as outfile:
        for entry in result:
            json.dump(entry, outfile, ensure_ascii=False)
            outfile.write('\n')
            
    #REPLACED
    result = []
    cnt=0
    for i,row in SP_DF.iterrows():
        item_removed = {"sentence_good":row['text'],"sentence_bad":row['replaced_sp'], "field": "disfluencies", 
                        "linguistics_term": "replaced_pause_"+LAN, "UID": "replaced_pause_"+exp_tag, "simple_LM_method": True,
                        "one_prefix_method": False, "two_prefix_method": False, 
                        "lexically_identical": False, "pair_id": str(cnt)}
        result.append(item_removed)
        cnt +=1
        
    with open(folder + 'replaced_pause_' + exp_tag + '.json', 'w', encoding='utf-8') as outfile:
        for entry in result:
            json.dump(entry, outfile, ensure_ascii=False)
            outfile.write('\n')

### REPEATS


In [25]:
### REPEATS

folder = './data_benchmark/disfl/'
if not os.path.exists(folder):
    os.makedirs(folder)

# FR
LAN = 'fr'
FILE_RP = 'data_corpus/french_repeats_500.txt'
SP_ITEMS= ['#']

REP_EXAMPLES = [u[:-1] for u in open(FILE_RP,'r', encoding = 'utf-8').readlines()]


REP_DF = pd.DataFrame(REP_EXAMPLES,columns=['text'])

for i in range(N_EXPS):
    REP_DF['moved_rep'] = REP_DF.apply(move_rep,axis=1)
    REP_DF['shuffled'] = REP_DF.apply(shuffle_utterance,axis=1)
    
    exp_tag = LAN+'_'+str(i)
    
    # SHUFFLED    
    result = []
    cnt=0
    for i,row in REP_DF.iterrows():
        item_shuffled = {"sentence_good":row['text'],"sentence_bad":row['shuffled'], "field": "disfluencies", 
                         "linguistics_term": "shuffled_"+LAN, "UID": "shuffled_"+exp_tag, "simple_LM_method": True, 
                         "one_prefix_method": False, "two_prefix_method": False, "lexically_identical": True, 
                         "pair_id": str(cnt)}
        result.append(item_shuffled)
        cnt +=1
            
    with open(folder + 'shuffled_repeat_' + exp_tag + '.json', 'w', encoding='utf-8') as outfile:
        for entry in result:
            json.dump(entry, outfile, ensure_ascii=False)
            outfile.write('\n')
        
    # MOVED    
    result = []
    cnt=0
    for i,row in REP_DF.iterrows():
        item_moved = {"sentence_good":row['text'], "sentence_bad":row['moved_rep'], "field": "disfluencies", 
                         "linguistics_term": "moved_rep_"+LAN, "UID": "moved_rep_"+exp_tag, "simple_LM_method": True,
                         "one_prefix_method": False, "two_prefix_method": False, 
                         "lexically_identical": True, "pair_id": str(cnt)}
        result.append(item_moved)
        cnt +=1
            
    with open(folder + 'moved_rep_' + exp_tag + '.json', 'w', encoding='utf-8') as outfile:
        for entry in result:
            json.dump(entry, outfile, ensure_ascii=False)
            outfile.write('\n')
            





In [26]:
REP_DF

,text,moved_rep,shuffled
0,c' était un bar avec euh un un écran ou #,c' était un bar avec avec euh un écran ou #,ou # bar écran un c' avec un un euh était
1,en fait elle elle était anglaise,anglaise en fait elle était anglaise,elle elle fait en était anglaise
2,style le le la dinde avec euh de la gelée à je...,style le la dinde avec euh euh de la gelée à j...,la avec style sais pas le gelée le à de je quo...
3,ou l' éducation # savait pas trop si c' était ...,ou l' éducation # savait pas trop si c' était ...,marqué a savait pendant éducation était quoi d...
4,eux ils restaient très très British quoi,eux eux ils restaient très British quoi,très restaient ils eux British très quoi
...,...,...,...
495,donc sur les tu vois les les bénéfi-,donc donc sur les tu vois les bénéfi-,les les sur tu bénéfi- vois les donc
496,mais ça me paraissait pas juste quoi de ouais ...,mais ça me paraissait pas juste quoi quoi de o...,eu- j' euh que ça mais fran- sortir paraissait...
497,du coup quand quand tu arrives je veux dire tu...,du coup quand tu arrives je veux dire tu as au...,du arrives tu * veux # quand dire tu quand je ...
498,et puis bon mo- moi ça me gêne pas le le truc,et puis bon mo- moi ça me gêne gêne pas le truc,le moi bon mo- le ça pas me et gêne truc puis


# DM

In [8]:
def replace_dm(row):
    right_as_list = row['right'].split()[1:]
    dm_candidates = [dm for dm in DMS if dm != row['dm']]
    random.shuffle(dm_candidates)
    return row['left'] + ' ' + dm_candidates[0] + ' ' + ' '.join(right_as_list)



In [12]:
folder = './data_benchmark/discourse/'
if not os.path.exists(folder):
    os.makedirs(folder)
    
# FR
LAN = 'fr'
FILE_DM_MONO = 'data_corpus/french_DM_mono.txt'
FILE_DM_DIAL = 'data_corpus/french_DM_dial.txt'
DMS = ['mais', 'donc', 'parce_que', 'enfin', 'alors','puis','bon','après','puisque','ben']


DM_EXAMPLES = [u.split('<$>') for u in open(FILE_DM_MONO,'r', encoding = 'utf-8').readlines()]


DM_DF = pd.DataFrame(DM_EXAMPLES,columns=['left','dm','right'])

for i in range(N_EXPS):
    DM_DF['text'] = DM_DF['left'] + ' '+  DM_DF['right']
    DM_DF['replaced_dm'] = DM_DF.apply(replace_dm,axis=1)
    DM_DF['shuffled'] = DM_DF.apply(shuffle_utterance,axis=1)
    
    exp_tag = LAN+'_'+str(i)
    
    #REPLACED
    result = []
    cnt=0
    for i,row in DM_DF.iterrows():
        item_replaced = {"sentence_good":row['text'],"sentence_bad":row['replaced_dm'], "field": "discourse", 
                         "linguistics_term": "replaced_"+LAN, "UID": "replaced_"+exp_tag, "simple_LM_method": True, 
                         "one_prefix_method": False, "two_prefix_method": False, "lexically_identical": False, 
                         "pair_id": str(cnt)}
        result.append(item_replaced)
        cnt +=1
            
    with open(folder + 'replaced_dm_' + exp_tag + '.json', 'w', encoding='utf-8') as outfile:
        for entry in result:
            json.dump(entry, outfile, ensure_ascii=False)
            outfile.write('\n')
        
    # SHUFFLED    
    result = []
    cnt=0
    for i,row in DM_DF.iterrows():
        item_moved = {"sentence_good":row['text'], "sentence_bad":row['shuffled'], "field": "discourse", 
                         "linguistics_term": "shuffled_"+LAN, "UID": "shuffled_"+exp_tag, "simple_LM_method": True,
                         "one_prefix_method": False, "two_prefix_method": False, 
                         "lexically_identical": True, "pair_id": str(cnt)}
        result.append(item_moved)
        cnt +=1
                    
    with open(folder + 'shuffled_dm_' + exp_tag + '.json', 'w', encoding='utf-8') as outfile:
        for entry in result:
            json.dump(entry, outfile, ensure_ascii=False)
            outfile.write('\n')